In [169]:
# ライブラリの読み込み
import os
import scipy as sp
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
# グラフをインラインで表示させる
%matplotlib inline

In [170]:
def preprocess(train_flg):
    # train set の場合の処理
    if train_flg:
        df=pd.read_csv('./train_set.csv')
        # 外れ値処理
        df.drop(df[df['PRICE']>=4.0e7].index,inplace=True)
        df.drop(df[df['PRICE']<=400].index,inplace=True)
        df.drop(df[df['LIVING_GBA']>4000].index,inplace=True)
        #リフォーム年が欠損している場合、築年数を入れる
        df.loc[df["YR_RMDL"].isnull(), "YR_RMDL"] = df[df["YR_RMDL"].isnull()]["AYB"]
        
        # 訓練用targetデータ
        df_y_cond = np.log(df[df['SOURCE']=='Condominium']['PRICE'])
        df_y_resi = np.log(df[df['SOURCE']=='Residential']['PRICE'])
    else:
        df=pd.read_csv('./test_set.csv')
        
    df_cond = df[df['SOURCE']=='Condominium']
    df_resi = df[df['SOURCE']=='Residential']
    
    # for Condominium
    # 使う変数を定義
    cols = ['LATITUDE','LONGITUDE', 'ROOMS','FIREPLACES','YR_RMDL','SALEDATE','LIVING_GBA']    
    df_cond = df_cond.loc[:,cols]
    # test, trainで共通の前処理
    # 築年数　→ 販売日 - 建築年　に変換
    df_cond.loc[:,'YR_RMDL'] = pd.to_datetime(df_cond['SALEDATE']).dt.year - df_cond['YR_RMDL'].values
    df_cond.drop('SALEDATE', axis=1, inplace=True)
    # 欠損値の置換
    df_cond.loc[df_cond['YR_RMDL'].isnull(), 'YR_RMDL'] = df['YR_RMDL'].median()
    
    #df_cond.loc[df_cond['YR_RMDL'],] = np.where(df_cond.loc[df_cond['YR_RMDL'].isnull(), df['AYB']], df_cond.loc[df_cond['YR_RMDL']])
    #df['AYB'.isnull(), 'AYB'] = df['AYB'].median()
    #df_cond.loc["YR_RMDL"] = np.where(df_cond.loc["YR_RMDL"].isnull(), df_cond.loc["AYB"], df_cond.loc["YR_RMDL"])
    # 異常値の変換
    df_cond.loc[df_cond['YR_RMDL']<0, 'YR_RMDL'] = 0
    # スケーリング
    stdscaler = StandardScaler()
    df_cond = stdscaler.fit_transform(df_cond)    
    
    
    # for Residential
    cols = ['LATITUDE','LONGITUDE', 'ROOMS','FIREPLACES','YR_RMDL','SALEDATE','GBA']    
    df_resi = df_resi.loc[:,cols].copy()
    # test, trainで共通の前処理
    # 築年数　→ 販売日 - 建築年　に変換
    df_resi.loc[:,'YR_RMDL'] = pd.to_datetime(df_resi['SALEDATE']).dt.year - df_resi['YR_RMDL'].values    
    df_resi.drop('SALEDATE', axis=1, inplace=True)
    # 欠損値の置換
    df_resi.loc[df_resi['YR_RMDL'].isnull(), 'YR_RMDL'] = df_resi['YR_RMDL'].median()
    # 異常値の変換
    df_resi.loc[df_resi['YR_RMDL']<0, 'YR_RMDL'] = 0
    #外れ値の処理
    #df_resi.drop(df_resi[df_resi['ROOMS']>=20].index, inplace=True)
    # スケーリング
    stdscaler = StandardScaler()
    df_resi = stdscaler.fit_transform(df_resi)    
    
    
    # train set の場合のoutput
    if train_flg:
        return df_resi,df_y_resi,df_cond,df_y_cond
    # test set の場合のoutput、kaggleへ提出するファイルの規定により、idも一緒に出力する
    else:
        return df_resi,df_cond,df['Id']

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
    
# cross validationのスコア表示
def display_score(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [171]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn import  preprocessing, linear_model
# train set
# 前処理の統合変数を使う
df_x_resi,df_y_resi,df_x_cond,df_y_cond = preprocess(train_flg=True)

reg_resi = RandomForestRegressor()
res_cond = RandomForestRegressor()
#reg_resi = LinearRegression()
#reg_cond = LinearRegression()
# linear regressionで検証
reg_resi.fit(df_x_resi, df_y_resi)
reg_cond.fit(df_x_cond, df_y_cond)

# 評価
# rmse(clf.predict(df_x),df_y)
# cross validationで評価
print('--------resi-----------')
scores = cross_val_score(reg_resi, df_x_resi, df_y_resi,
                            scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_score(rmse_scores)

print('--------cond-----------')
scores = cross_val_score(reg_cond, df_x_cond, df_y_cond,
                            scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_score(rmse_scores)

C:\Users\world\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3248: DtypeWarning: Columns (17,18,19,20,21,22,23,32,33,34,36,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\world\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


--------resi-----------
Scores: [0.74938211 0.64867648 0.56143321 0.50857377 0.68921696 0.59758866
 0.56369182 0.64891035 0.62269539 0.71391247]
Mean: 0.630408120516672
Standard deviation: 0.07102897108175439
--------cond-----------
Scores: [0.90083512 0.64026772 0.56388953 0.83792589 0.72748856 0.49955235
 0.72943059 0.54528109 0.52727986 0.59476707]
Mean: 0.6566717777152188
Standard deviation: 0.13012461504820785


In [172]:
# test set
# 前処理の統合変数を使う
df_x_resi,df_x_cond,df_test_id = preprocess(train_flg=False)

resi_y_pred = np.exp(reg_resi.predict(df_x_resi))
cond_y_pred = np.exp(reg_cond.predict(df_x_cond))

pred_df = pd.DataFrame(np.concatenate((resi_y_pred,cond_y_pred),axis=0), index=df_test_id, columns=["PRICE"])
pred_df.to_csv('./output.csv', header=True, index_label='Id')

C:\Users\world\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3248: DtypeWarning: Columns (17,18,19,20,21,22,23,32,33,34,36,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
